<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed data in the dataSource, the import scopes for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

# Basic Initialization

In [0]:
#!import "../../Constants/CalculationEngine"

In [0]:
#!eval-notebook "../../Initialization/InitSystemorphBaseToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

# Import of parsed data

In [0]:
Debug.Enable = true;

In [0]:
//ProjectionCount = 5;

In [0]:
var fileName = "../../Files/TransactionalData/NominalCashflows_CH_2020_12.csv";
await Import.FromFile(fileName)
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

## Debug the above if fails

In [0]:
// var extension = System.IO.Path.GetExtension(fileName);
// var stream = await Project.FileStorage.ReadAsync(fileName);
// var dataSet = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;
// var args = (await GetArgsAndCommitPartitionAsync<PartitionByReportingNodeAndPeriod>(dataSet, DataSource)) with {ImportFormat = ImportFormats.Cashflow};

In [0]:
// var workspace = Workspace.CreateNew();
// await ParseActualsToWorkspaceAsync(dataSet, args, workspace, DataSource)

# Args

## Primary Args

In [0]:
var reportingNode = "CH";
var year = 2021;
var month = 3;
var importFormat = ImportFormats.Actual;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

## Secondary Args

In [0]:
var allArgs = await GetAllArgsAsync(args, DataSource, ImportFormats.Cashflow);
allArgs

In [0]:
// var workspaceToCompute = Workspace.CreateNew();
// workspaceToCompute.InitializeFrom(DataSource);

In [0]:
// await ComputeAsync(args, Workspace, workspaceToCompute, true)

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();

# Create universe and Identities

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [0]:
var ret = universe.GetScopes<CumulatedNominalBE>(identities)
                .SelectMany(x => x.PresentValues)
                .Select(x => new {Value= x.Value, 
                                  Id = x.Identity.Id, 
                                  AmoutType = x.Identity.AmountType,
                                  EstimateType = x.Identity.EstimateType,
                                  AccidentYear = x.Identity.AccidentYear});
ret.ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities, storage).ToScopes<ComputeAllScopes>().SelectMany(x => x.CalculatedIfrsVariables).ToArray();